Import Libraries

In [207]:
from pyvis.network import Network
import networkx as nx
import random
from igraph import Graph

## Create Helper Functions

### Create paper function

In [208]:
# recursive function that will traverse the nodes
def createPaper(network, authors, probStop):
    '''
    Will take network, list of authors, and probStop as input
    '''
    currAuthorID = authors[-1]
    newNeighbors = set(network.neighbors(currAuthorID)).difference(set(authors))

    # base condition: stop at node if probStop hit or there are no new neighbors to traverse
    if random.random() < probStop or len(newNeighbors) == 0:
        return
    
    # create list reprsenting probabilities for the neighboring nodes of the current coauthor
    probs = []
    for neighbor in newNeighbors:
        nData = network.get_edge_data(currAuthorID, neighbor)
        probs.extend([neighbor] * nData["weight"])

    # Select coauthor from neighbors probabilities list
    coauthorID = random.choice(probs)

    # update all edges of coauthors to this new author
    for author in authors:
        # if there is not an edge, create one
        if not network.has_edge(author, coauthorID) and author != coauthorID:
            network.add_edge(author, coauthorID, weight=0, width=1)
        newWeight = network.get_edge_data(author, coauthorID)["weight"] + 1
        #network.update(edges=[ (author, coauthorID, {"weight": newWeight, "width": newWeight//2}) ])
        network.update(edges=[ (author, coauthorID, {"weight": newWeight}) ])

    # call function recursively with coauthor
    authors.append(coauthorID)
    createPaper(network, authors, probStop)


### Get New Community and Color
Function will return a new community value and color for new communities to use

In [209]:
def getNewCC(network):
    colors = {"red", "blue", "green", "orange", "purple", "yellow"}
    print(max(dict(network.nodes.data('label')).values()))
    newComm = max(dict(network.nodes.data('label')).values()) + 1
    return newComm, "red"

def getCommNodes(network, communityNum):
    '''Returns a list of nodes associated with the community'''
    nodes = []
    for nodeID, comm in network.nodes.data('label'):
        if comm == communityNum:
            nodes.append(nodeID)
    return nodes
    

### Split community function

In [210]:
def splitCommunity(network, nodes):
    '''
    Function will take the networkx network as input and the list of nodes in the community
        It will then test if it should split the community or not
    Returns nothing, will update the network
    '''
    # split into two communities
    subGraph = network.subgraph(nodes)
    newGraph = Graph.from_networkx(subGraph)

    # create subgraph and split
    clusters = newGraph.community_leading_eigenvector(clusters=2)

    # recreate network
    nt = Network()
    # populates the nodes and edges data structures
    nt.from_nx(subGraph)
    nt.show('docs/models/testsub.html')

    # compare unweighted modularity of new communities to the initial, return if there should not be change in community structure
    # Q: modularity of just the partition or the whole graph with the new partition?
    if newGraph.modularity(set(subGraph.nodes())) > clusters.modularity or len(clusters) != 2:
        return

    # update the colors and group name of the nodes in the smaller sub-community
    # must know all the groups and community names and pick different ones
    newComm, color = getNewCC(network)
    index = 1 if len(clusters[1]) < len(clusters[0]) else 0
    for node in clusters[index]:
        network.update(nodes=[(node, {"label": newComm, "color": color})])
    # update the papers? Need a data structure of the papers
    

Define initial parameters

In [211]:
# define time steps
timeSteps = 30

# Probabilities
# probability that you generate new author
probNewAuthor = 0.5
# probability that you stop at a given node
probStop = 0.7
# probability that a split event occurs
pd = 0.9

# define initial scholars, will be in form (id, scholarField, color)
scholarField = 0
nodeID = 0


Create Model

In [212]:
network = nx.Graph()
network.add_node(nodeID, label=scholarField, color="red")
# go through time steps, add new scholar and paper at each step
for i in range(1, timeSteps):

    # Choose first author, either new scholar or random choice
    currNodes = list(network.nodes())
    authors = [random.choice(currNodes)]

    # with probability, add new author to network set as main author with a coauthor
    if random.random() < probNewAuthor:
        # generate author and field
        nodeID += 1
        author = nodeID

        # generate random coauthor from currNodes,
        coauthorID = random.choice(currNodes)

        # add node with field being the co-author's field
        scholarField = network.nodes[coauthorID]["label"]
        network.add_node(author, label=scholarField, color="red")
        network.add_edge(author, coauthorID, weight=1, width=1)

        # update authors list
        authors = [author, coauthorID]

    # Add new paper, calling function
    createPaper(network, authors, probStop)

    # split random discipline with prob pd
    if random.random() < pd:
        commNodes = getCommNodes(network, 0)
        splitCommunity(network, commNodes)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


Display Network

In [213]:
nt = Network()
# populates the nodes and edges data structures
nt.from_nx(network)
print(network.edges.data())
nt.show('docs/models/modularity.html')

[(0, 1, {'weight': 2, 'width': 1}), (0, 2, {'weight': 2, 'width': 1}), (0, 3, {'weight': 2, 'width': 1}), (0, 8, {'weight': 1, 'width': 1}), (0, 7, {'weight': 2, 'width': 1}), (0, 11, {'weight': 2, 'width': 1}), (1, 6, {'weight': 1, 'width': 1}), (2, 4, {'weight': 1, 'width': 1}), (2, 7, {'weight': 2, 'width': 1}), (2, 8, {'weight': 1, 'width': 1}), (4, 5, {'weight': 1, 'width': 1}), (4, 9, {'weight': 4, 'width': 1}), (4, 13, {'weight': 1, 'width': 1}), (6, 10, {'weight': 2, 'width': 1}), (7, 8, {'weight': 3, 'width': 1}), (7, 11, {'weight': 2, 'width': 1}), (7, 12, {'weight': 1, 'width': 1}), (8, 12, {'weight': 1, 'width': 1}), (8, 11, {'weight': 1, 'width': 1}), (9, 13, {'weight': 1, 'width': 1}), (11, 12, {'weight': 1, 'width': 1})]


### Calculate modularity using iGraph

In [214]:
# newGraph = Graph.from_networkx(network)
# clusters = newGraph.community_leading_eigenvector(clusters=2)
# # check if modularity of clusters is greater than whole
# print(clusters)

# # loop and update cluster colors
# for node in clusters[0]:
#     network.update(nodes=[(node, {"label": "g1", "color": 1})])
# for node in clusters[1]:
#     network.update(nodes=[(node, {"label": "g2", "color": 100})])
# print(set(network.nodes()))
# print(f'test mod1: {newGraph.modularity(set(network.nodes()))}')
# print(f'test mod2: {clusters.modularity}')
# splitCommunity(network, [0, 1, 3, 4, 5, 9, 10, 12])

In [215]:
# recreate network
nt = Network()
# populates the nodes and edges data structures
nt.from_nx(network)
print(network.edges.data())
nt.show('docs/models/modularity.html')

[(0, 1, {'weight': 2, 'width': 1}), (0, 2, {'weight': 2, 'width': 1}), (0, 3, {'weight': 2, 'width': 1}), (0, 8, {'weight': 1, 'width': 1}), (0, 7, {'weight': 2, 'width': 1}), (0, 11, {'weight': 2, 'width': 1}), (1, 6, {'weight': 1, 'width': 1}), (2, 4, {'weight': 1, 'width': 1}), (2, 7, {'weight': 2, 'width': 1}), (2, 8, {'weight': 1, 'width': 1}), (4, 5, {'weight': 1, 'width': 1}), (4, 9, {'weight': 4, 'width': 1}), (4, 13, {'weight': 1, 'width': 1}), (6, 10, {'weight': 2, 'width': 1}), (7, 8, {'weight': 3, 'width': 1}), (7, 11, {'weight': 2, 'width': 1}), (7, 12, {'weight': 1, 'width': 1}), (8, 12, {'weight': 1, 'width': 1}), (8, 11, {'weight': 1, 'width': 1}), (9, 13, {'weight': 1, 'width': 1}), (11, 12, {'weight': 1, 'width': 1})]
